# Deep Reinforcement Learning:

*Deep reinforcement learning* introduces the use of deep neural networks to solve reinforcement learning problems. For example, in regular Q-learning, we relied on using an algorithm that computed a complete table of optimal Q-values. For environments with a massive state space, like chess, computing this table is impossible. The use of a neural network lets us approximate the Q-values instead. 

## Policy Learning

Policy learning involves trying to learn the optimal policy $\pi$ rather than trying to learn the optimal value function. There are *evolution* strategies to policy learning and there is the *policy gradients* approach to policy learning.

In policy learning, we consider a family of policies: $\pi_\theta : S \to A$, determined by  a neural network with weight parameters $\theta$. Here we're using a network that maps states in the input layer into actions in the output layer, and we wish to train the weights of the network towards a configuration that would correspond to the optimal policy.

The *fitness* of a policy $\pi_\theta$ is the value function of the initial state $s_0$, ie. the average discounted reward received by the agent if we follow policy $\pi_\theta$:

$$
    \text{fitness}(\pi_\theta) = V^{\pi_\theta}(s_0)=E_{\pi_\theta}(r_{\text{total}})
$$

$E_{\pi_\theta}(r_{\text{total}})$ is the expectation of the total reward if we follow policy $\pi_\theta$.


Note: the stoachastic policy $\pi_\theta$ outputs a probability distribution rather than a specific value. So
$$
\pi_\theta (a_t|s_t)
$$ is the probability that the policy chooses action $a_t$ __from__ state $s_t$.

A deterministic policy, on the other hand, outputs the same action for the given state all the time.

### Hill Climbing &mdash; Evolution Strategy:

Suppose we have a network $\pi_\theta$ with weights $\theta$ which is the policy function for a particular game.

At any given time, we have a certain set of weights in this network corresponding to a *champion*, ie. a particular set of network weights that produce the current best player. Initially, the champion weights would just be a set of weights initialised to $0$, so $\theta_{\text{champ}} = 0$.

For each trial, we produce a *mutant* by adding random noise to the champion weights: 
$$\theta_{\text{mutant}} = \theta_{\text{champ}} + \text{Gaussian noise},$$ drawn from a normal distribution with some fixed standard deviation $\sigma$.

The mutant is evaluated against the champion on the same task. If the mutant performs better than our current champion, then we apply the following update rule on the champion weights:

$$
    \theta_{\text{champ}} := \underbrace{(1-\alpha)\theta_{\text{champ}}}_{\text{Retained champ weights}} + \alpha \theta_{\text{mutant}},
$$

where $\alpha = 0.1$, for example. If the champion does better than the mutant, then no updating is done.

### Policy Gradients &mdash; Episodic:

Policy gradients is an alternative to the evolution strategy. It uses *gradient ascent* rather than adding random noise weights to produce a better policy.

Suppose we have an agent playing an episodic game. So it takes the sequence of actions $a_1, a_2, ..., a_t, ... a_m$ and it receives $r_{\text{total}}$ at the end.

Since we don't have method for determining *which* actions contributed to the reward, we attribute the contribution of each action $a_t$ to total reward equally.

If $r_{\text{total}}$ is high, the weights of the policy network are updated so that the agent is more likely to take the same sequence of actions, $a_1, a_2, ..., a_t, ... a_m$, in the future. We do this by training the network to increase the output activation value of each action $a_t$, or in other words, the log probability of each action:

$$
    \sum_{t=1}^{m} \log \big( \pi_{\theta} (a_t | s_t) \big) = \log \big( \prod_{t=1}^m \pi_\theta (a_t | s_t) \big), \tag{1}
$$
where $\pi_{\theta} (a_t | s_t)$ is the probability that the policy will choose action $a_t$, given it is at the state $s_t$, and $\prod_{t=1}^m \pi_\theta(a_t | s_t)$ is the probability of the policy network choosing the sequence of actions $a_1, a_2, ..., a_t, ... a_m$. The policy network can compute the probability distribution of actions that it currently thinks is best, $\pi_{\theta} (a_t | s_t)$, by applying the $\texttt{softmax}$ activation function at the output layer.

If $r_{\text{total}}$ is negative, the weights of the policy network are changed so that the agent is less likely to choose the same actions in the future. We do this by decreasing $(1)$.



If we want to get the network to increase the log probabilities in $(1)$, we can simply multiply $(1)$ by $r_{\text{total}}$ and take the derivative with respect to every weight $\theta_1, \theta_2, \ldots$:

$$
    \triangledown_\theta r_{\text{total}} \sum_{t=1}^m \log \big( \pi_\theta (a_t | s_t) \big) =  r_{\text{total}} \sum_{t=1}^m \triangledown_\theta \log \big( \pi_\theta (a_t | s_t) \big)  \tag{2}
$$

where $\triangledown_\theta \log \big( \pi_\theta (a_t | s_t) \big)$ is the vector of gradients of the log probability with respect to each individual weight $\theta_1,  \theta_2, \ldots, \theta_n$.

In general, the symbol $\triangledown_y$ means $(\frac{\partial}{\partial y_1}, \frac{\partial}{\partial y_2}, \ldots, \frac{\partial}{\partial y_n})$. The subscript tells you which variable's vector components we are differentiating with respect to.

#### Reward/Punishment baseline:
What if the game always gives positive reward values $r_{\text{total}} > 0$, for example between 4 and 5? Suppose that $r_{\text{total}}=4$ corresponds to a loss. Because the reward is positive, the network will always reward actions even if they contributed to a poorer outcome.

If the game always gives positive rewards $r_{\text{total}}$, then we can substitute $r_{\text{total}}$ in equation $(2)$ to be $r_{\text{total}} - b$, where $b$ is the reward/punishment *baseline*. We could set $b=4.5$, for example, if the range of values is the interval $[4, 5]$. Any total reward greater than this baseline would lead to the sequence of actions taken being reward while any total reward less than this baseline would lead to the sequence of actions taken being punished.

#### REINFORCE Algorithm:

The following algorithm has successfully been applied in the earlier reinforcement learning models (1992), eg. for playing Pong, where the image of the game is passed into a convolutional neural network for the agent's policy.  

- For each episode:
    - Run the agent through the episode fully and collect the:
        - sequence of states $s_t$
        - actions $a_t$ taken 
        - total reward obtained $r_{\text{total}}$ 
    - For $t=1$ to $m$, the number of time steps in the current trial:
        $$\theta := \theta + \eta\underbrace{(r_{\text{total}} - b)}_{\text{Reward baseline}} \cdot \underbrace{\triangledown_\theta \log \big ( \pi_\theta (a_t | s_t) \big)}_{\text{Analogous to } \frac{\partial E}{\partial \theta}} \tag{3}$$

### Policy Gradients &mdash; Non-Episodic:

Suppose now that we're dealing with a non-episodic game where rewards can be received at different timesteps throughout the game, rather than only at the completion of the game (eg. Pacman).

Suppose you keep a fixed policy $\pi_\theta$ and you play multiple games, each time recording which states $s \in S$ you were in. Every policy $\pi_\theta$ determines a probability distribution $\rho_{\pi_\theta} (s)$ of states $s \in S$:

$$
    \rho_{\pi_\theta} (s) = \sum_{t \geq 0} \gamma^t \text{prob}_{\pi_\theta, t}(s),
$$

where $\text{prob}_{\pi_\theta, t}(s)$ is the probability that after starting in initial state $s_0$, you reach state $s$ after $t$ actions. The value $\rho_{\pi_\theta} (s)$ is the number of times we expect to visit state $s$ if we use policy $\pi_\theta$, discounted by $\gamma^t$.
- This probability distribution is just theoretical, it's not something we compute since the state space could be unfathomably large. The idea is that a *good policy* network $\pi_\theta$ would have a higher probability of landing you in better game states. Eg. if you were training a Pacman model, then a good policy would have higher probabilities for states where the white dots are mostly eaten. So for 'good' states $s$, we have a higher $\rho_{\pi_\theta} (s)$ value

We can then define the *fitness* of a policy network as:

$$
    \text{fitness} (\pi_\theta) = \sum_{s \in S} \big( \rho_{\pi_\theta}(s)\cdot \big( \sum_{a \in A} Q^{\pi_\theta} (s, a) \cdot \pi_\theta (a | s) \big) \big), \tag{4}
$$
and the aim of the policy network is to maximise this fitness quantity.

Differentiating $(4)$ with respect to every weight parameter $\theta_1, \theta_2, \ldots$ in the policy network, we have


$$
\begin{align}
    \triangledown_\theta \text{fitness} (\pi_\theta) &= \sum_{s \in S} \big( \rho_{\pi_\theta}(s)\cdot \big( \sum_{a \in A} Q^{\pi_\theta} (s, a) \cdot \triangledown_\theta \pi_\theta (a | s) \big) \big) \\
    &= \sum_{s \in S} \big( \rho_{\pi_\theta}(s)\cdot \big( \sum_{a \in A} Q^{\pi_\theta} (s, a) \cdot \triangledown_\theta \pi_\theta (a | s) \big) \\
    &= \sum_{s \in S} \big( \rho_{\pi_\theta}(s)\cdot \big( \sum_{a \in A} Q^{\pi_\theta} (s, a) \cdot \pi_\theta (a | s) \cdot \frac{\triangledown_\theta \pi_\theta (a | s)}{\pi_\theta (a | s)} \big) \\
    &= \sum_{s \in S} \big( \rho_{\pi_\theta}(s)\cdot \big( \sum_{a \in A} Q^{\pi_\theta} (s, a) \cdot \pi_\theta (a | s) \cdot \triangledown_\theta \log (\pi_\theta (a | s)) \big) \\
    &= E_{\pi_\theta}\big( Q^{\pi_\theta} (s, a) \cdot \triangledown_\theta \log(\pi_\theta (a | s)) \big). \tag{5}
\end{align}
$$

Note that the gradient of $ \rho_{\pi_\theta}(s)$ and $Q^{\pi_\theta} (s, a)$ are very hard to determine, so only the gradient of $\pi_\theta (a | s)$ is computed

### Actor-Critic Paradigm:

An actor network and a critic network are both learnt together. The actor-critic paradigm combines value-function learning with policy learning.

__Critic network__ &mdash; in $(5)$, we can't easily find a good estimate for $Q^{\pi_\theta}(s, a)$, so we instead train a network on weights $w$ which determines a Q-function $Q_w$ that aims to approximate/converge towards $Q^{\pi_\theta}$.
- The Q function network $Q_W$ with weight parameters $W$ takes in a state and action in the input layer and predicts Q-values in the output layer 

__Actor network__ &mdash; simultaneously, we are also training a network on weights $\theta$ that determines policy $\pi_\theta$.
- The policy network $\pi_\theta$ with parameters $\theta$ takes in a state in the input layer and predicts the next action to take

The actor makes a decision on what action to take, based on the state, and the critic takes in the action as input, along with state, and assesses how good that chosen action was.

The actor-critic method of training tends to take less training time than policy gradient methods.

#### Actor-Critic Algorithm:

- For each episode:
    - We are initially on starting state $s_0$. First we select the initial action $a_0$ from the probability distribution $\pi(a|s_0)$
    - For each time step $t$:
        - Get reward at this timestep $r_t$
        - Get the next state $s_{t+1}$
        - Get the next action $a_{t+1}$, from the probability distribution $\pi(a | s_{t+1})$
        - $\frac{dE}{dQ}=-\big( r_t + \gamma Q_w (s_{t+1}, a_{t+1}) - Q_w (s_t, a_t) \big)$
        - $\underbrace{\theta := \theta + \eta_\theta \cdot Q_w (s_t, a_t) \cdot \triangledown_\theta \log(\pi_\theta (a_t|s_t))}_{\text{Policy gradient update rule. See (3)}}$
        - $w := w - \eta_w \cdot \frac{dE}{dQ} \cdot \triangledown_w Q_w (s_t, a_t)$ 

### Deep Q-Learning Network for Atari Games:

With traditional Q-learning, we create a lookup table of Q-values to help us determine what action to take on each state. With deep Q-learning, we introduce a neural network to approximate the Q-values instead (since it's impractical to store massive lookup tables, and we need to generalise to unseen situations.

With a lookup table, Q-learning is guaranteed to converge eventually, however for most applications there are too many states to be reasonably stored in a lookup table.

Instead, we have $Q_w(s, a)$ determined by network weights $w$, which get updated so as to minimise the error:

$$
    E = \big( \underbrace{r_t + \gamma \max_{b} Q_w (s_{t+1}, b)}_{\text{'Correct'}} - \underbrace{Q_w(s_t, a_t)}_{\text{Predicted}} \big)^2,
$$
which is just the mean-squared error function.

Note that this is not supervised learning, so we don't exactly have a true optimal Q-value to compare the predicted value with. Instead, we just estimate the true optimal Q-value as: $\texttt{true Q-value} = r_t + \gamma \max_{b} Q_w (s_{t+1}, b)$.

Of course, when the network begins training, the value of $\max_{b} Q_w (s_{t+1}, b)$ is completely wrong. The reward $r_t$, however, is correct, so the network will slowly build up better estimations in a similar fashion to the tabular Q-learning algorithm.



<table>
    <tr>
        <td width="45%">            
            <img src="images/deep-q-learning-vs-lookup.png" width="100%">
            <p style="text-align: center;">The input to the deep Q network is the current state of the game and the output is the vector of Q-values associated with each of chosen possible actions the agent can take.</p>
        </td>
        <td>
            <img src="images/deep-q-network-overview.png" width="100%">
            <p style="text-align: center;">
                The deep-Q network accepts a state vector, for example, a screenshot of the current game frame, and outputs its predictions of the Q-values for each action that the agent can take. No activation function is used at the output layer since we want the raw untransformed output values.
            </p>
            <p style="text-align: center;">
                The agent takes the next action to be the action corresponding to $\texttt{argmax}$ of the output layer.
            </p>
        </td>
    <tr>
</table>


<table>
    <tr>
        <td>            
            <img src="images/q-learning-network-atari.png" width="100%">
            <p style="text-align: center;">
                Deep Q networks don't introduce any new neural network architecture. It usually just takes a standard CNN network, plus fully connected layers and uses this network to approximate the optimal Q-values, ie. the best action to take, given the input state.
            </p>
        </td>
        <td>
            <img src="images/q-learning-network-atari-skeleton.png" width="100%">
            <p style="text-align: center;">
                The input consists of 4 screenshots of consecutive frames of the game state. If we were training the network to play Atari breakout, for example, then doing this lets the network determine the direction the ball is moving. A single still frame doesn't indicate the direction of the ball, but a sequence of still frames does.
            </p>
        </td>
    <tr>
</table>



#### Deep Q-Learning with Experience Replay:

*Experience replay* is a technique we use during deep Q network training.

We store the agent's 'experiences' at each time step in a dataset called the __*replay memory*__.

The agent's experience at time $t$ is the tuple $e_t=(s_t, a_t, r_{t+1}, s_{t+1})$ &mdash; where $s_t$ is the state the agent was in, $a_t$ is the action the agent took, $r_{t+1}$ is the reward the action received from taking action $a_t$, and $s_{t+1}$ is the state where the agent ended up after taking action $a_t$.

This tuple summarises the agent's *experience* at time $t$. The replay memory is a buffer of size $n$ that stores a sequence of experiences $[e_0, e_1, \ldots, e_{n-1}]$. The size is finite, so everytime a new experience is to be added, the oldest experience is popped to make room for the new experience.

Experience replay is when we randomly sample experiences from the replay memory buffer and training the network on the selected experiences.

Why not just train the network regularly with a sequence of experiences? By randomly sampling individual experiences to train the network on, we're removing the temporal correlation between game situations.
- Consider the task of training a CNN image classifier. Training the network on a sequence of 1000 bird images, followed by 1000 car images, etc. performs poorly compared to training on a randomised sequence of images.

*Prioritised replay* &mdash; instead of randomly picking games to train on, each with equal probability, we pick games according to the greatest error value: 
$$
    E= |r_t + \gamma \max_{b} Q_w (s_{t+1}, b) - Q_w (s_t, a_t)|
$$
- This makes the network place greater emphasis on learning from situations where the Q-value was 'most surprising'. Eg. if you were training a Pacman model, then eating a ghost would give a surprisingly large reward. This situation would be prioritised in experience replay.


### Resources:
- <a href="https://medium.com/deep-math-machine-learning-ai/ch-13-deep-reinforcement-learning-deep-q-learning-and-policy-gradients-towards-agi-a2a0b611617e">Deep reinforcement learning comprehensive</a>
